# Toggle switch

In [ ]:
from output_helper import *
%matplotlib inline

In [ ]:
with xr.open_dataset("output/ts/output_t50000.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    slice_vec = np.array([0, 0])
    P_full = lr_sol.fullDistribution()
    P_marginal = lr_sol.marginalDistributions()
    P_sliced = lr_sol.slicedDistributions(slice_vec)

In [ ]:
with open("scripts/reference_solutions/ts_ode_ref.npy", "rb") as f:
    P_full_ode = np.load(f, allow_pickle=True)
    P_marginal_ode = np.load(f, allow_pickle=True)
    P_sliced_ode = np.load(f, allow_pickle=True)
    P_best_approximation_ode = np.load(f, allow_pickle=True)
    n_ode = np.load(f, allow_pickle=True)

Additional parameters:

In [ ]:
# Figure 1 and 2
idx = np.arange(2)
mesh = [grid.bin[j] * np.arange(grid.n[j]) + grid.liml[j] for j in idx]
mesh_ode = [np.arange(el.shape[0]) for el in P_marginal_ode[-1]]

# Figure 4
t = 500
time_step = 10
snapshot = 100

## Figure 1
One-dimensional marginal distributions for $t = 500$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2.5))
plotP1Dmult(axs, [P_marginal, P_marginal_ode[-1]], [mesh, mesh_ode], ["DLR approx.", "exact"], idx, "MD", stats=True)
# axs.flat[0].legend()
fig.suptitle("\\textbf{{rank \\textit{{r}}}} $\mathbf{{=5}}$", y=0.92, fontsize="medium")
fig.tight_layout()
fig.savefig('plots/ts_fig1.pgf')

## Figure 2
Sliced distributions for $x_0 = (0,\,0)$, $t = 500$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
grid_ref = [range(el.shape[0]) for el in P_sliced_ode[-1]]
plotP1Dmult(axs, [P_sliced, P_sliced_ode[-1]], [mesh, mesh_ode], ["DLR approx.", "exact"], idx, "S", stats=True)
fig.tight_layout()

## Figure 3
Full probability distribution for $t = 500$

In [ ]:
xx1, xx2 = np.meshgrid(
    np.arange(grid.n1[0]), np.arange(grid.n2[0]))
fig, axs = plt.subplots(1, 2, figsize=(6, 3))
plt.setp(axs.flat, xlim=[0, 30], ylim=[0, 30], xlabel="$x_1$", ylabel="$x_2$", aspect="equal")
plotP2D(axs, [P_full.reshape((grid.n1[0], grid.n2[0])), P_full_ode[-1].reshape(n_ode)], [xx1, xx2], ["DLR approximation", "exact"])
fig.tight_layout()
fig.savefig('plots/ts_fig2.pgf')

Maximal error for the full probability distribution:

In [ ]:
print(np.amax(np.abs(P_full - P_full_ode[-1])))

## Figure 4
Calculate the 2-norm error for the DLR approximation and for the truncated SVD of the reference solution ('best approximation')

In [ ]:
times = np.arange(0, t + time_step, time_step)
err = np.zeros((times.size, 2))

for ts, tp in enumerate(times):
    filename = "output/ts/output_t" + str(int(tp * snapshot)) + ".nc"
    with xr.open_dataset(filename) as ds:
        grid = GridInfo(ds)
        lr_sol = LRSol(ds, grid)
        P_full = lr_sol.fullDistribution()
    # err[ts, 2] = np.linalg.norm((P_full - P_full_ode[ts]))
    # filename = "output/ts_tau0.02/output_t" + str(int(10 * tp * snapshot)) + ".nc"
    # with ds.open_dataset(filename, "r") as ds:
    #     grid = GridInfo(ds)
    #     lr_sol = LRSol(ds, grid)
    #     P_full = lr_sol.fullDistribution()
    err[ts, 0] = np.linalg.norm((P_full - P_full_ode[ts]))
    err[ts, 1] = np.linalg.norm((P_best_approximation_ode[ts] - P_full_ode[ts]))

fig, ax = plt.subplots(figsize=(5, 3))
# ax.plot(times, err[:, 2], 'r-', label="DLR approx., $\\tau=0.2$")
ax.plot(times, err[:, 0], 'b-.', label="DLR approx., $\\tau=0.01$")
ax.plot(times, err[:, 1], 'k--', label="best-approximation")
ax.set_xlabel("$t$")
ax.set_ylabel("2-norm error")
ax.legend()
fig.tight_layout()
fig.savefig('plots/ts_fig3.pgf')